<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Illustration-of-the-system" data-toc-modified-id="Illustration-of-the-system-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Illustration of the system</a></span></li><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test runs</a></span></li><li><span><a href="#Tests" data-toc-modified-id="Tests-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tests</a></span></li></ul></div>

# Description

Notebook to illustrate how population models work

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, interact_manual

from statsmodels.tsa.stattools import acf

from scipy.signal import find_peaks

In [ ]:
from RBmodel.analysis import get_cycle_stats

from RBmodel import cell_models

from RBmodel import load_utils

In [ ]:
import itertools

# Illustration of the system

In [ ]:
params = cell_models.DEFAULT_PARAMS.copy()

In [ ]:
params['transition'] = "size"

In [ ]:
params

In [ ]:
N_cells=100 # number of cells in the population

T = 10000 # number of timesteps

pop = cell_models.population(N_cells, params=params)
pop.grow(T)

In [ ]:
results = pop.gather_results()

In [ ]:
_, ax = plt.subplots(2, 2, figsize=(10, 8))

for (i,j) in itertools.product(range(2), repeat=2):
    idx = T / 4 * j + i * T/2
    
    ax[i,j].hist(results["M"].loc[idx], bins=np.linspace(0, 2, 20));
    ax[i,j].grid()
    ax[i,j].set_title(f"time = {idx}")

In [ ]:
flat_M = results["M"].values.flatten().round(decimals=2)

flat_ph = results["phase"].values.flatten()

vals_M = np.sort(np.unique(flat_M))
rat_g1g2 = []
for m in vals_M: 
    idx = np.where(flat_M==m)[0]
    phases = flat_ph[idx]
    G1_G2 = np.sum(phases=='G1')/len(phases)
    rat_g1g2.append(G1_G2)

In [ ]:
plt.plot(vals_M, 1-np.array(rat_g1g2))
plt.xlabel("M")
plt.ylabel("Proportion of cells in G2")
plt.grid()

In [ ]:
plt.rcParams.update({'font.size': 22})
_, ax = plt.subplots(2, 3, figsize=(10, 8))
color='gray'
s = 2

for cell in pop.cells:
    phase_durations, stats = load_utils.get_phase_durations(cell)

    ax[0,0].scatter(stats['birth'], stats['growth'][0], color=color, s=s)
    ax[0,1].scatter(stats['birth'], stats['length'][0], color=color, s=s)
    ax[0,2].scatter(
        stats['birth'], 
        (np.array(stats['growth'][0])-1) * np.array(stats['birth']), 
        color=color, 
        s=s
    )
    ax[1,0].scatter(stats['birth'], stats['RB_G1'][0], color=color, s=s)
    ax[1,1].scatter(stats['birth'], stats['RB_G1'][1], color=color, s=s)
    ax[1,2].scatter(stats['birth'], stats['RBc_G1'][0], color=color, s=s)
    
ax[0, 0].grid()
ax[0, 0].set_xlabel("M at birth")
ax[0, 0].set_ylabel("M_G1S/M_birth")

ax[0, 1].grid()
ax[0, 1].set_xlabel("M at birth")
ax[0, 1].set_ylabel("G1 Length")

ax[0, 2].grid()
ax[0, 2].set_xlabel("M at birth")
ax[0, 2].set_ylabel("M_G1S - M_birth")

ax[1, 0].grid()
ax[1, 0].set_xlabel("M at birth")
ax[1, 0].set_ylabel("RB amount at birth")

ax[1, 1].grid()
ax[1, 1].set_xlabel("M at birth")
ax[1, 1].set_ylabel("RB amount at division")

ax[1, 2].grid()
ax[1, 2].set_xlabel("M at birth")
ax[1, 2].set_ylabel("RB concentration at birth")

plt.tight_layout()